This seems to be the lowest level of ML ANN architecture I'll be working at. In this lab/hw I am going to be at to solve a problem. Most room for modifications! 

Problem 
-------
Set up a basic neural net using TensorFlow following the example from lecture.

Use the same data and testing parameters from the Breast Cancer exercise in Hw 1 to make the same predictions with this new NN.



Updates 3-12
-----------

I've put most of the code in place, but I need to repeat this with Siraj's version of this with MNIST to see what he does with training and testing data, along with the epochs, batch size, etc. 
Run a print statment on the following variables I'm unsure of: 

next_batch
and more!


Huge error message at the end which includes a ton of warnings, gonna look at this again soon!

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import cross_validation

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import numpy as np

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)



In [8]:
data = pd.read_csv('cancer.dat', header = None, names = ['ID', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli','Mitoses','Class'],skiprows=[23,40,139,145,158,164,235,249,275,292,294,297,315,321,411,617]) 
X = data.values[:,1:10]
y = pd.get_dummies(data['Class']).values
X_train, X_test, y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)

In [5]:
X = tf.convert_to_tensor(X)

In [7]:
inputX = X_train.s

tensorflow.python.framework.ops.Tensor

In [11]:
#Set parameters
learning_rate = .01 #how fast we update the weights, .01 is a known decent size
training_iteration = 30
batch_size = 10
display_step = 2

Going to more or less pull Siraj's MNIST Tensorflow here, make edits where neccessary to turn it into a breast cancer predictor. 

In [5]:
# TF graph input
#Need to identify proper dimensions, let's do that now (later)
x = tf.placeholder("float", [None, 9]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 2]) # 0-9 digits recognition => 10 classes



In [6]:
W = tf.Variable(tf.zeros([9, 2]))
b = tf.Variable(tf.zeros([2]))
 #for visualization in tensorboard
with tf.name_scope("Wx_b") as scope:
    # Construct a linear model
    model = tf.nn.relu(tf.matmul(x, W) + b) # Softmax

w_h = tf.summary.histogram("weights", W)
b_h = tf.summary.histogram("biases", b)


# More name scopes will clean up graph representation
with tf.name_scope("cost_function") as scope:
    # Minimize error using cross entropy
    # Cross entropy, this was used for MNIST but should suffice for BC since also classificaiton. 
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # Create a summary to monitor the cost function
    tf.summary.scalar("cost_function", cost_function)
    
with tf.name_scope("train") as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)



In [7]:
# Initializing the variables
init = tf.initialize_all_variables()

# Merge all summaries into a single operator
merged_summary_op = tf.summary.merge_all()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [15]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Change this to a location on your computer
    summary_writer = tf.summary.FileWriter('/Users/noahkasmanoff/MachLearn/hw3/', graph_def=sess.graph_def)

    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(len(X_train)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys =  next_batch(batch_size,data=X_train,labels=y_train)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Tuning completed!")

    # Test the model, still in session! 
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: y_test}))

InvalidArgumentError: Nan in summary histogram for: weights
	 [[Node: weights = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](weights/tag, Variable/read)]]

Caused by op 'weights', defined at:
  File "/Users/noahkasmanoff/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/noahkasmanoff/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177,

In [10]:
X_train.new

100